## MNIST Training and Prediction with SageMaker Chainer

[MNIST](http://yann.lecun.com/exdb/mnist/), the "Hello World" of machine learning, is a popular dataset for handwritten digit classification. It consists of 70,000 28x28 grayscale images labeled in 10 digit classes(0 to 9). This tutorial will show how to train on MNIST using SageMaker Chainer in [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk).

In [ ]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

## Download MNIST datasets

We can use chainer built-in get_mnist() method to download, import and structure MNIST dataset. It may take several minutes to run this cell for the first time.

In [ ]:
import chainer

try:
    os.makedirs('data/train')
    os.makedirs('data/test')
except FileExistsError:
    print('Directories already exist!')

train, test = chainer.datasets.get_mnist()

## Parse, save, and upload the data

Now we split each of training dataset and testing dataset to images and labels. Then it's convenient for us to save training data and testing data to npz files, a format that's popular in chainer world, to local filesystem. Then we can use `sagemaker.Session.upload_data` to upload the data to an S3 location used for training. The return value `inputs` identifies this S3 location.

In [ ]:
import os

import numpy as np

train_images = np.array([data[0] for data in train])
train_labels = np.array([data[1] for data in train])
test_images = np.array([data[0] for data in test])
test_labels = np.array([data[1] for data in test])

np.savez('data/train/train.npz', images=train_images, labels=train_labels)
np.savez('data/test/test.npz', images=test_images, labels=test_labels)

train_input = sagemaker_session.upload_data(path=os.path.join('data', 'train'), key_prefix='notebook/chainer/mnist')
test_input = sagemaker_session.upload_data(path=os.path.join('data', 'test'), key_prefix='notebook/chainer/mnist')

## Prepare user script for training and prediction

We need to provide a user script that can run on the SageMaker platform. The script is essentially the same as one you would write for local training, except that a function `train` that returns a trained `chainer.Chain` model is required. By default, this model is saved to disk as an npz file named `model.npz`.
For prediciton, this script also requires a function `model_fn` that loads the `chainer.Chain` saved from training. When SageMaker calls your `train` and `model_fn` functions, it will pass in arguments that describe the training environment.

### TODO: Add link to README for user script writing here.

The below user script used in our tutorial is adapted from [chainer mnist example](https://github.com/chainer/chainer/blob/master/examples/mnist/train_mnist.py)

In [ ]:
!cat 'chainer_mnist_single_machine.py'

## Create SageMaker chainer estimator

To train MNIST, let's construct a `sagemaker.chainer.estimator.Chainer` estimator. A quick explanation for some configurable arguments here:

`entry_point`: The user script SageMaker runs for training and prediction.

`train_instance_count`: The number of SageMaker instances for training. Since we only do single machine training in this tutorial, it should be 1.

`train_instance_type`: The type of SageMaker instances for training. We choose `ml.c5.xlarge` for cpu training. If you want gpu training, you could choose other instance type accordingly. See [Amazon SageMaker ML Instance Types](https://aws.amazon.com/sagemaker/pricing/instance-types/)

`hyperparameters`: The hyper-parameters defined in the user script. In this tutorial, `epochs`, `batch_size` and `frequency` can be configured and passed.

In [ ]:
from sagemaker.chainer.estimator import Chainer

chainer_estimator = Chainer(entry_point='chainer_mnist_single_machine.py', role=role,
                            sagemaker_session=sagemaker_session,
                            train_instance_count=1, train_instance_type='ml.c5.xlarge',
                            hyperparameters={'epochs': 16, 'batch_size': 128})

## Train on MNIST data in S3

After we've constructed our Chainer object, we can fit it using the MNIST data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our user script can simply read the data from disk.

In [ ]:
chainer_estimator.fit({'train': train_input, 'test': test_input})

In [ ]:
## Deploy model to endpoint

After training, we deploy the model to an endpoint using the Chainer estimator object.

In [ ]:
predictor = chainer_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## Predict Hand-Written Digit

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a variable named 'data' in this notebook, which we can then pass to the Chainer predictor.

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

Now let's see if your writing can be recognized!

In [ ]:
image = np.array(data, dtype=np.float32)
prediction = predictor.predict(image)
predicted_label = prediction.argmax(axis=1)[0]
print('What you wrote is: {}'.format(predicted_label))

We can also get some random test images in MNIST.

In [ ]:
import random

import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(test_images.shape[0] - 1), num_samples)
images, labels = test_images[indices], test_labels[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')

Now let's see if we can make correct predictions.

In [ ]:
prediction = predictor.predict(images)
predicted_label = prediction.argmax(axis=1)
print('The predicted labels are: {}'.format(predicted_label))